# **Proyecto ML:** ***Riesgo de obesidad***

<p align="center">
  <img src="https://www-rockandpop-cl.cdn.ampproject.org/i/s/www.rockandpop.cl/wp-content/uploads/2019/10/obesidad-y-sobrepeso-como-prevenir.jpg" alt="Texto alternativo" width="600" height="300">
</p>

### **1.Introducción**

### **2.Librerías**

In [1]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Visualización
import matplotlib.pyplot as plt
import seaborn as sns

# Modelos
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, cross_val_predict
from sklearn.preprocessing import OneHotEncoder, LabelEncoder, MinMaxScaler, StandardScaler
from sklearn.model_selection import GridSearchCV, StratifiedKFold

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier, AdaBoostClassifier, HistGradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
#from xgboost import XGBClassifier
#from lightgbm import LGBMClassifier
#from catboost import CatBoostClassifier



from sklearn.metrics import balanced_accuracy_score, make_scorer,classification_report
pd.set_option('display.max.column',None)

### **3.Carga de datos**

In [ ]:
train=pd.read_csv('')
test=pd.read_csv('')

### **4.Primera exploración**

**4.1.Primera visualización del DataFrame**

In [ ]:
train.head()

In [ ]:
train.tail()

**4.2.Estructura del DataFrame**

In [ ]:
train.shape

**4.3.Clasificación de variables**

In [ ]:
train.info()

**4.4.Revisión de duplicados**

In [ ]:
train.duplicated().sum()

**4.5.Revisión de valores faltantes**

In [ ]:
train.isna().sum()

**4.6.Resumen estadístico general del DataFrame**

In [ ]:
train.describe(exclude='object').round(2)

In [ ]:
train.describe(include='object')

**4.7.Descripción general de los datos**

### **5.EDA**

**5.1.Cardinalidad y tipo de variables**

Haremos un estudio sobre la cardinalidad, para observar el número de valores únicos de cada variable
y ver si encontramos algo significativo para nuestro estudio.

In [ ]:
cardinalidad = pd.DataFrame({
    'Columna': train.columns,
    'Tipo_Variable':['Discreta' if x == 'int64' else 'Continua' if x=='float64' else 'Nominal' for x in train.dtypes],
    'Cardinalidad%':[round((train[col].nunique())/len(train)*100,2) for col in train.columns]})

cardinalidad

**5.2.Análisis univariante**

**5.3.Análisis bivariante**

**6.Feature Engineer**

**7.División del dataset**

**8.Preprocesamiento**

**9.Cross validation + Base-Lines**

**Entranamiento**

**Predicción**

**Validación del modelo**

**Optimización para el modelo**

**Predicción con test**